In [1]:
pip install jupyter_contrib_nbextensions && jupyter contrib nbextension install --user

Note: you may need to restart the kernel to use updated packages.


[I 19:28:06 InstallContribNbextensionsApp] jupyter contrib nbextension install --user
[I 19:28:06 InstallContribNbextensionsApp] Installing jupyter_contrib_nbextensions nbextension files to jupyter data directory
[I 19:28:06 InstallContribNbextensionsApp] Installing c:\users\brena\anaconda3\lib\site-packages\jupyter_contrib_nbextensions\nbextensions\addbefore -> addbefore
[I 19:28:06 InstallContribNbextensionsApp] Up to date: C:\Users\Brena\AppData\Roaming\jupyter\nbextensions\addbefore\addbefore.yaml
[I 19:28:06 InstallContribNbextensionsApp] Up to date: C:\Users\Brena\AppData\Roaming\jupyter\nbextensions\addbefore\icon.png
[I 19:28:06 InstallContribNbextensionsApp] Up to date: C:\Users\Brena\AppData\Roaming\jupyter\nbextensions\addbefore\main.js
[I 19:28:06 InstallContribNbextensionsApp] Up to date: C:\Users\Brena\AppData\Roaming\jupyter\nbextensions\addbefore\readme.md
[I 19:28:06 InstallContribNbextensionsApp] - Validating: ok
[I 19:28:06 InstallContribNbextensionsApp] Installing c

In [2]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [3]:
# selenium을 백그라운드로 실행하기 위해 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')

In [4]:
# 우리가 데이터를 가져올 웹 페이지입니다.
my_opgg_url = 'https://www.op.gg/summoner/userName=midplayer+or+afk'
driver = webdriver.Chrome("/Users/Brena/Desktop/chromedriver", options= options)

<ipython-input-4-11792aabe17f>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/Brena/Desktop/chromedriver", options= options)


In [5]:
# 데이터를 가져올 페이지로 이동합니다.
driver.get(my_opgg_url)

In [6]:
queue_select = Select(driver.find_element_by_css_selector('select.SelectMatchTypes'))

<ipython-input-6-4d9555680c1d>:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  queue_select = Select(driver.find_element_by_css_selector('select.SelectMatchTypes'))


In [7]:
queue_select.select_by_value('aram')
time.sleep(2)

In [8]:
# '더 보기' 버튼이 활성화되어있으면(리스트의 끝까지 도달하지 않았으면) 계속 클릭하여 이전 게임을 모두 로딩합니다.
while True:
    try:
        driver.find_element_by_css_selector('.GameMoreButton').click()
        # 게임 로딩, html 코드 변경까지 여유 시간을 1초 가집니다.
        time.sleep(1)
        
    # 에러가 나면(페이지에서 '더 보기' 버튼이 없을 경우) while문을 탈출합니다.
    except Exception as e:
        pass
        break

<ipython-input-8-523ee828ac39>:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector('.GameMoreButton').click()


In [9]:
# 게임이 모두 로딩된 페이지의 html 코드를 faker_total_html로 저장합니다.
my_total_html = driver.page_source

# selenium이 제어하는 크롬을 종료합니다.
driver.quit()

In [10]:
# html 형식에 맞춰 파싱(parsing; 추후 이용하기 쉽도록 쪼개기)합니다.
my_total_soup = BeautifulSoup(my_total_html, 'html.parser')

In [11]:
# 전체 html 코드 중 우리가 원하는 selector를 만족하는 것만 가져오기
my_total_games_html = my_total_soup.select('div.GameItemList div.GameItemWrap')

# 그렇게 가져온 html 코드의 element 개수 == 끝까지 로딩된 모든 게임 수 출력
total_game_len = len(my_total_games_html)
print(total_game_len)

297


In [12]:
# 결과가 들어갈 빈 리스트를 만듭니다.
my_champions = []
my_KDA = []
my_results = []
my_multikill = []
my_KP = []

In [13]:
for i in range(total_game_len):
    if my_total_games_html[i].select_one('div.KDA div.MultiKill span.Kill ') != None:
        my_multikill.append(list(my_total_games_html[i].select_one('div.KDA div.MultiKill span.Kill ').
                                               stripped_strings)[0])
    else:
        my_multikill.append('None')
    my_champions.append(list(my_total_games_html[i].select_one('div.ChampionName').
                                               stripped_strings)[0])
    my_results.append(list(my_total_games_html[i].select_one('div.GameStats div.GameResult').
                                               stripped_strings)[0])
    my_KDA.append(list(my_total_games_html[i].select_one('div.KDA div.KDARatio span.KDARatio ').
                                               stripped_strings)[0])
    my_KP.append(list(my_total_games_html[i].select_one('div.Stats div.CKRate').
                                               stripped_strings)[0])

In [14]:
# DataFrame으로 변환 후 출력
my_total_df = pd.DataFrame([my_champions,
                            my_results, my_KDA,
                            my_multikill, my_KP],
                               index = ['champion','result','KDA', 'multikill','KP']).T
my_total_df

,champion,result,KDA,multikill,KP
0,탐 켄치,승리,4.57:1,None,킬관여 82%
1,아칼리,패배,1.76:1,더블킬,킬관여 45%
2,브라움,패배,2.70:1,None,킬관여 64%
3,아무무,패배,2.60:1,None,킬관여 72%
4,요네,승리,2.33:1,트리플킬,킬관여 58%
...,...,...,...,...,...
292,잭스,패배,2.31:1,더블킬,킬관여 73%
293,카이사,패배,4.22:1,더블킬,킬관여 69%
294,룰루,패배,2.75:1,None,킬관여 73%
295,브랜드,승리,3.11:1,None,킬관여 61%


In [30]:
my_total_df["multikill"].value_counts()

None    138
더블킬     116
트리플킬     40
쿼드라킬      3
Name: multikill, dtype: int64